In [3]:
import os
from adversarial_attacks import test
from utils import util_functions as uf
import matplotlib.pyplot as plt
import torch
import argparse
import cv2
print("Started...")

models = [
#    "alexnet", 
    "resnet18", 
#    "softalexnet", 
    "softresnet18"
]

iters = [32768, 38868, 46241, 55009, 92582, 110218, 185264]
epsilons = [0.0001, 0.001, 0.01, 0.1]

BATCH_SIZE = 150
MIN_EXAMPLES = 50
train_len = "long"
iter_prefix = "best_iter"

pos = len(iter_prefix.split("_"))

def list_dirs(_model):
    p = "checkpoints/{}_trained/{}".format(train_len, _model)
    ll = [x for x in os.listdir(p) if iter_prefix in x]
    ll = [int(x.split("_")[pos]) for x in ll]
    ll = sorted(ll)
    return ll

def get_image_both_sizes(dataset="places", val=True):
    image_batch, labels, orig_paths, names, \
    cats, orig_images, catlist = test.get_batch("resnet18",
                                                batch_size=BATCH_SIZE)
    paths = orig_paths[0]
    base = uf.find_child_dir_path("datasets")
    basepath_val = os.path.join(base, dataset, "val")
    basepath_train = os.path.join(base, dataset, "train")
    image_batch_alex, labels_alex, paths = \
        uf.get_image_as_batch(paths=tuple(paths),
                              val_data_folder=basepath_val,
                              train_data_fold=basepath_train,
                              val=val, model="alexnet",
                              batch_size=BATCH_SIZE)
    image_batch_alex, labels_alex = image_batch_alex.cuda(), labels_alex.cuda()
    lbls_sorted, _ = torch.sort(labels)
    lbls_alex_sorted, _ = torch.sort(labels_alex)
    assert torch.equal(lbls_sorted, lbls_alex_sorted)
    return image_batch_alex, image_batch, labels, orig_paths


image_batch_alex, image_batch, labels, orig_paths = None, None, None, None
attempt = 0

#find image that is classified correctly for all models and iters
while attempt < 50:
    print("attempt = {}".format(attempt))
    image_batch_alex, image_batch, labels, orig_paths = get_image_both_sizes()
    attempt += 1
    failed = False
    for mod in models:
        print("*** model {} ***".format(mod))
        for it in iters:
            print("it {}".format(it))
            
            model = uf.get_model_arg(mod, it, longtrain=train_len=="long")
            model.eval()
            model.cuda()
            batch = image_batch_alex if "alex" in mod else image_batch
            out = model(batch.cuda())
            predcat = torch.argmax(out, dim=1)
            correct_imgs = torch.flatten(torch.nonzero(predcat == labels))
            batch = batch[correct_imgs]

            print("Remained {} imgs, need at least {}".format(
                batch.shape[0], MIN_EXAMPLES))

            if batch.shape[0] < MIN_EXAMPLES:
                print("failed!")
                failed = True
                break
        if failed is True:
            break
    if failed is False:
        break


print("Batched picked!")

if attempt >= 50:
    print("Didn't find an image with correct classification!")
    exit()

rows = 1
cols = 2
figsize = (cols * 8, rows * 5)
fig, axs = plt.subplots(rows, cols, figsize=figsize)
mid = (fig.subplotpars.right + fig.subplotpars.left) / 2
plt.subplots_adjust(left=None, bottom=None, right=None, top=1.2,
                    wspace=None, hspace=None)
plt.suptitle("# imgs {}".format(image_batch.shape[0]), x=mid, y=1)

print("Starting attacks...")

for i, mod in enumerate(models):
    accs = []
    ax = axs[i]
    for it in iters:
        epsilons, robust_accuracy, raw, clipped, \
            is_adv, (predcat, adv_predcat, label) = test.run_attacks(mod, it,
                image_batch if "alexnet" not in mod else image_batch_alex,
                labels, attack_name="PGD", longtrain=train_len=="long",
                epsilons=epsilons)

        accs.append(robust_accuracy)

    ax.set_title(mod)
    for j, ep in enumerate(epsilons):
        ax.plot(iters, [x[j] for x in accs], label="it {}".format(it))

plt.show()
plt.close()


[16034596.log: 10-03 14:18:02] p51148 {/data/vision/torralba/dissect/nadiia/adversarial_research/utils/util_functions.py:142} DEBUG - Data dir path /data/vision/torralba/dissect/nadiia/adversarial_research/datasets/places/val
[16034596.log: 10-03 14:18:02] p51148 {/data/vision/torralba/dissect/nadiia/adversarial_research/utils/util_functions.py:143} DEBUG - Creating dataset with batch_size 150, num_workers=0
[16034596.log: 10-03 14:18:02] p51148 {/data/vision/torralba/dissect/nadiia/adversarial_research/utils/util_functions.py:79} DEBUG - Creating transformer with crop_size=224, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
[16034596.log: 10-03 14:18:02] p51148 {/data/vision/torralba/dissect/nadiia/adversarial_research/utils/util_functions.py:124} DEBUG - Loading dataset from folder /data/vision/torralba/dissect/nadiia/adversarial_research/datasets/places/val


Started...
attempt = 0
*** model resnet18 ***
it 32768


RuntimeError: CUDA out of memory. Tried to allocate 460.00 MiB (GPU 0; 23.65 GiB total capacity; 18.47 GiB already allocated; 83.00 MiB free; 19.45 GiB reserved in total by PyTorch)

In [2]:
ll1 = list_dirs("softresnet18")
ll2 = list_dirs("softalexnet")
ll3 = list_dirs("alexnet")
ll4 = list_dirs("resnet18")
print(ll3)
print(ll4)
mi = min([
    # ll1[-1],
    # ll2[-1],
    ll3[-1],
    ll4[-1]
])
print(mi)

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.
[0, 1, 509, 624, 861, 1118, 1348, 1448, 1622, 1722, 1948, 2048, 2335, 2435, 2796, 2896, 3344, 3444, 3996, 4771, 5693, 5793, 6789, 9642, 9742, 11585, 13677, 16284, 19384, 19484, 23070, 23170, 27454, 27554, 32668, 32768, 38868, 38968, 46241, 65536, 77836, 77936, 110118, 130972, 185264]
[0, 5, 6, 8, 10, 54, 64, 76, 91, 108, 128, 152, 181, 204, 215, 256, 262, 304, 331, 362, 412, 431, 509, 512, 609, 624, 724, 761, 861, 924, 1118, 1218, 1348, 1448, 1622, 1722, 1948, 2335, 2435, 2796, 2896, 3344, 3444, 3996, 4096, 4771, 4871, 5693, 5793, 6789, 6889, 8092, 9642, 9742, 11485, 13677, 13777, 19384, 27454, 32768, 38868, 65536, 92582, 155872, 524188]
185264
